In [11]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [19]:
# 다음 카페 검색 페이지로 이동
driver = setWebdriver()
driver.get('https://search.daum.net/search?w=fusion&col=cafe&q=%EB%85%B8%EC%9D%B8+%EB%B6%80%EC%96%91&DA=STC&sd=20240301000000&ed=20240325235959&period=u')

In [20]:
# 제목과 URL을 저장할 리스트 초기화
title_list = []  
url_list = []     
  
# 카페 글 크롤링
while(True):
    try:
        for i in range(1,11): # 페이지 수만큼 반복
            # 글의 제목과 URL 추출 # //*[@id="twcColl"]/div[2]/c-container/c-card[1]/div/c-doc-web/div/div[1]/c-title/strong/a
            title = driver.find_element(By.XPATH,'//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]'.format(i)).text # 제목
            url = driver.find_element(By.XPATH, '//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]/c-title'.format(i)).get_attribute('data-href') # URL 추출
    
            title_list.append(title) # 제목 추가
            url_list.append(url) # URL 추가
        
        driver.find_element(By.CLASS_NAME, 'btn_next').click() # 다음 페이지로 이동
    except:
        break

In [ ]:
# 제목과 URL을 저장할 리스트 초기화
title_list = []  
url_list = []     
  
# 카페 글 크롤링
while(True):
    try:
        for i in range(1,11): # 페이지 수만큼 반복
            # 글의 제목과 URL 추출 # //*[@id="twcColl"]/div[2]/c-container/c-card[1]/div/c-doc-web/div/div[1]/c-title/strong/a
            title = driver.find_element(By.XPATH,'//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]'.format(i)).text # 제목
            url = driver.find_element(By.XPATH, '//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]/c-title'.format(i)).get_attribute('data-href') # URL 추출
    
            title_list.append(title) # 제목 추가
            url_list.append(url) # URL 추가
        
        driver.find_element(By.CLASS_NAME, 'btn_next').click() # 다음 페이지로 이동
    except:
        break

---

### Data preprocessing

In [26]:
import re
text = "core1 core core883core"
text = "core1"
re.findall(r'\bcore', text)

['core']

In [ ]:
text = "Wow! 999, This is a wonderful place"
w_text = re.findall(f'Wow! 999. This is')

In [27]:
file_path = '../data/'
file_name = 'daumCafe_crawling(노인 부양, 20230101-20231231).pkl'
with open(file_path + file_name, 'rb') as f:
    data = pickle.load(f)
print(data)

                                                 title  \
0                                            노인 부양의 시대   
1               부양가족연금_60대 '노인 가장'국민연금 年 27만6000원 더 타감   
2                  "생산인구 1명이 노인 1명 부양할 판…'죽음의 계곡' 될 것"   
3                        1 욕먹어 가면서 노인 복지정책의 개선을 바라는 이유   
4                                 생산인구 1명이 노인 1명 부양할 판   
..                                                 ...   
145                                       [플랜75]이라는 영화   
146  노인은 무엇으로 사는가 - 감사의 마음으로 사는 것이다 / 복사골 아저씨님 / 좋은글방1   
147                                       [플랜75]이라는 영화   
148                                       [플랜75]이라는 영화   
149                                       [플랜75]이라는 영화   

                                                   url  \
0    https://cafe.daum.net/dicapoetry/1aSh/24409?q=...   
1    https://cafe.daum.net/knoub/9RXe/1158?q=%EB%85...   
2    https://cafe.daum.net/djh43/KAK0/3898?q=%EB%85...   
3    https://cafe.daum.net/oldSoLa/fonP/7?q=%EB%85%...   
4    https://

In [ ]:
docs['comment'] = docs['comment'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
docs['comment']